In [1]:
import pandas as pd

school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

merged_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name","school_name"])

merged_df.head(10)

C:\Users\ccrom\anaconda3\envs\PythonData\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


## District Summary

In [2]:
#Determine total schools in district 
total_schools = len(merged_df["school_name"].unique())

#Determine total students in the district
total_students = merged_df["student_name"].count()

#Determine total budget of the district
total_budget = merged_df["budget"].unique().sum()

#Determine mean math score in district
average_math = merged_df["math_score"].mean()

#Determine mean reading score in district 
average_reading = merged_df["reading_score"].mean()

#Determine how many students passed math
passing_math = merged_df[(merged_df["math_score"] >= 70)].count()["student_name"]
percentage_passing_math = (passing_math/total_students) *100

#Determine how many students passed reading
passing_reading =  merged_df[(merged_df["reading_score"] >= 70)].count()["student_name"]
percentage_passing_reading = (passing_reading/total_students) *100

#Determine how many students passed both reading and math
passing_both = merged_df[(merged_df["math_score"] >= 70) & (merged_df["reading_score"] >=70)].count()["student_name"]
percentage_passing_both = (passing_both / total_students) * 100

In [3]:
#Create a Dataframe
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools], 
    "Total Students":[total_students], 
    "Total Budget":[total_budget], 
    "Average Math Score":[average_math], 
    "Average Reading Score":[average_reading], 
    "% Passing Math":[percentage_passing_math], 
    "% Passing Reading":[percentage_passing_reading], 
    "% Overall Passing":[percentage_passing_both]
})

#Format Data 
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)

#Show
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [4]:
#Set og data set to school name and groupy by school name set to groups by school
groups_by_school = merged_df.set_index("school_name").groupby(["school_name"])

#Get school type from school data, and save to school type
school_types = school_data_df.set_index("school_name")['type']

#Count students in each school 
students_per_school = groups_by_school["Student ID"].count()

#Get budget values
school_budget = school_data_df.set_index("school_name")["budget"]

#Get per student budget from school data
per_student_budget = school_data_df.set_index("school_name")["budget"]/school_data_df.set_index("school_name")["size"]

#Getting average math and reading
average_math = groups_by_school["math_score"].mean()
average_reading = groups_by_school["reading_score"].mean()

#Getting average passing, reading, and overall
passing_math = merged_df.loc[(merged_df["math_score"] >= 70)].groupby("school_name")["Student ID"].count()/students_per_school * 100
passing_reading = merged_df.loc[(merged_df["reading_score"] >= 70)].groupby("school_name")["Student ID"].count()/students_per_school * 100
passing_both = merged_df.loc[(merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)].groupby("school_name")["Student ID"].count()/students_per_school * 100

#Create dataframe
school_summary_df = pd.DataFrame({
    "School Type":school_types,
    "Total Students": students_per_school,
    "Total School Budget": school_budget, 
    "Per Student Budget": per_student_budget, 
    "Average Math": average_math, 
    "Average Reading": average_reading, 
    "% Passing Math": passing_math, 
    "% Passing Reading":passing_reading,
    "% Overall Passing": passing_both
})


#Format Data 
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df["Average Math"] = school_summary_df["Average Math"].map("{:.2f}".format)
school_summary_df["Average Reading"] = school_summary_df["Average Reading"].map("{:.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:.2f}%".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:.2f}%".format)

#Show
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Top 5 Performing Schools by Overall Passing

In [5]:
#Sort values ex. from sorting ex
top_five = school_summary_df.sort_values("% Overall Passing", ascending = False)

top_five.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## ## Bottom 5 Performing Schools by Overall Passing

In [6]:
#get bottom 5 values
bottom_five = top_five.tail()
bottom_five = bottom_five.sort_values("% Overall Passing")

bottom_five.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

In [7]:
#Conditional to get math scores by grade by school 
nineth = merged_df[(merged_df["grade"] == "9th")].groupby("school_name").mean()["math_score"]
tenth = merged_df[(merged_df["grade"] == "10th")].groupby("school_name").mean()["math_score"]
eleventh = merged_df[(merged_df["grade"] == "11th")].groupby("school_name").mean()["math_score"]
twelfth = merged_df[(merged_df["grade"] == "12th")].groupby("school_name").mean()["math_score"]

#Create Dataframe
math_average_df = pd.DataFrame({
    "9th Grade": nineth, 
    "10th Grade":tenth, 
    "11th Grade":eleventh, 
    "12th Grade": twelfth
})

#Format
math_average_df["9th Grade"] = math_average_df["9th Grade"].map("{:.2f}%".format)
math_average_df["10th Grade"] = math_average_df["10th Grade"].map("{:.2f}%".format)
math_average_df["11th Grade"] = math_average_df["11th Grade"].map("{:.2f}%".format)
math_average_df["12th Grade"] = math_average_df["12th Grade"].map("{:.2f}%".format)

#Show
math_average_df



,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Scores by Grade

In [8]:
#Same thing

nineth = merged_df[(merged_df["grade"] == "9th")].groupby("school_name").mean()["reading_score"]
tenth = merged_df[(merged_df["grade"] == "10th")].groupby("school_name").mean()["reading_score"]
eleventh = merged_df[(merged_df["grade"] == "11th")].groupby("school_name").mean()["reading_score"]
twelfth = merged_df[(merged_df["grade"] == "12th")].groupby("school_name").mean()["reading_score"]

#Create Data Frame
reading_average_df = pd.DataFrame({
    "9th Grade": nineth, 
    "10th Grade":tenth, 
    "11th Grade":eleventh, 
    "12th Grade": twelfth
})

#Format
reading_average_df["9th Grade"] = reading_average_df["9th Grade"].map("{:.2f}%".format)
reading_average_df["10th Grade"] = reading_average_df["10th Grade"].map("{:.2f}%".format)
reading_average_df["11th Grade"] = reading_average_df["11th Grade"].map("{:.2f}%".format)
reading_average_df["12th Grade"] = reading_average_df["12th Grade"].map("{:.2f}%".format)

#Show
reading_average_df


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

In [9]:
#Create bins, they're exclusive yeah?
bins = [0, 584, 629, 644, 680]

#Create labels
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Create a column and cut merged df into per student
merged_df["spending_student"] = pd.cut(merged_df['budget']/merged_df['size'], bins, labels = labels)

#group by bins
spending = merged_df.groupby("spending_student")

#re-do calculations, inserting new groupby 
average_math = spending["math_score"].mean()
average_reading = spending["reading_score"].mean()
passing_math = merged_df.loc[(merged_df["math_score"] >= 70)].groupby("spending_student")["Student ID"].count()/spending["student_name"].count() * 100
passing_reading = merged_df.loc[(merged_df["reading_score"] >= 70)].groupby("spending_student")["Student ID"].count()/spending["student_name"].count() * 100
passing_both = merged_df.loc[(merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)].groupby("spending_student")["Student ID"].count()/spending["student_name"].count() * 100

#Create dataframe
spending = pd.DataFrame({ 
    "Average Math": average_math, 
    "Average Reading": average_reading, 
    "% Passing Math": passing_math, 
    "% Passing Reading":passing_reading,
    "% Overall Passing": passing_both
})

#format
spending["Average Math"] = spending["Average Math"].map("{:.2f}".format)
spending["Average Reading"] = spending["Average Reading"].map("{:.2f}".format)
spending["% Passing Math"] = spending["% Passing Math"].map("{:.2f}%".format)
spending["% Passing Reading"] = spending["% Passing Reading"].map("{:.2f}%".format)
spending["% Overall Passing"] = spending["% Overall Passing"].map("{:.2f}%".format)

#show
spending


,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
spending_student,,,,,
<$585,83.36,83.96,93.70%,96.69%,90.64%
$585-630,79.98,82.31,79.11%,88.51%,70.94%
$630-645,77.82,81.30,70.62%,82.60%,58.84%
$645-680,77.05,81.01,66.23%,81.11%,53.53%


## Scores by School Size

In [10]:
#same thing, create bins
bins = [0, 1000, 1999, 5000]

#create labels
labels = ["Small (<1000)", "Medium (1000-1999)", "Large (2000-5000)"]

#create a new columns sizes splits size row into bins..
merged_df["sizes"] = pd.cut(merged_df["size"], bins, labels=labels)


#merge by school size
school_size = merged_df.groupby("sizes")

#do calcs
average_math = school_size["math_score"].mean()
average_reading = school_size["reading_score"].mean()
passing_math = merged_df.loc[(merged_df["math_score"] >= 70)].groupby("sizes")["Student ID"].count()/school_size["student_name"].count() * 100
passing_reading = merged_df.loc[(merged_df["reading_score"] >= 70)].groupby("sizes")["Student ID"].count()/school_size["student_name"].count() * 100
passing_both = merged_df.loc[(merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)].groupby("sizes")["Student ID"].count()/school_size["student_name"].count() * 100

#Dataframe
school_size = pd.DataFrame({ 
    "Average Math": average_math, 
    "Average Reading": average_reading, 
    "% Passing Math": passing_math, 
    "% Passing Reading":passing_reading,
    "% Overall Passing": passing_both
})

#format
school_size["Average Math"] = school_size["Average Math"].map("{:.2f}".format)
school_size["Average Reading"] = school_size["Average Reading"].map("{:.2f}".format)
school_size["% Passing Math"] = school_size["% Passing Math"].map("{:.2f}%".format)
school_size["% Passing Reading"] = school_size["% Passing Reading"].map("{:.2f}%".format)
school_size["% Overall Passing"] = school_size["% Overall Passing"].map("{:.2f}%".format)

#show
school_size


,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
sizes,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,90.14%
Medium (1000-1999),83.37,83.87,93.62%,96.77%,90.62%
Large (2000-5000),77.48,81.20,68.65%,82.13%,56.57%


## Scores By Shool Type

In [12]:
#group by type
score_type= merged_df.groupby("type")

#calcs
average_math = score_type["math_score"].mean()
average_reading = score_type["reading_score"].mean()
passing_math = merged_df.loc[(merged_df["math_score"] >= 70)].groupby("type")["Student ID"].count()/score_type["student_name"].count() * 100
passing_reading = merged_df.loc[(merged_df["reading_score"] >= 70)].groupby("type")["Student ID"].count()/score_type["student_name"].count() * 100
passing_both = merged_df.loc[(merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)].groupby("type")["Student ID"].count()/score_type["student_name"].count() * 100

#create dataframe
score_type = pd.DataFrame({ 
    "Average Math": average_math, 
    "Average Reading": average_reading, 
    "% Passing Math": passing_math, 
    "% Passing Reading":passing_reading,
    "% Overall Passing": passing_both
})

#format
score_type["Average Math"] = score_type["Average Math"].map("{:.2f}".format)
score_type["Average Reading"] = score_type["Average Reading"].map("{:.2f}".format)
score_type["% Passing Math"] = score_type["% Passing Math"].map("{:.2f}%".format)
score_type["% Passing Reading"] = score_type["% Passing Reading"].map("{:.2f}%".format)
score_type["% Overall Passing"] = score_type["% Overall Passing"].map("{:.2f}%".format)

#show
score_type

,Average Math,Average Reading,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.41,83.90,93.70%,96.65%,90.56%
District,76.99,80.96,66.52%,80.91%,53.70%
